In [1]:
!python --version

Python 3.12.12


In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
from google.cloud import bigquery
from google.colab import drive

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, GATConv
from torch_geometric.loader import NeighborLoader
import networkx as nx
import community as community_louvain
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, roc_curve)
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [4]:
torch.__version__

'2.9.0+cu126'

In [5]:
print(torch.cuda.get_device_name())
print(torch.version.cuda)
x = torch.randn(1).cuda()
print(x)

NVIDIA A100-SXM4-40GB
12.6
tensor([-0.7052], device='cuda:0')


In [6]:
torch.device('cuda:0')

device(type='cuda', index=0)

In [7]:
project_id = 'cs467-project-479503'
bq_clinet = bigquery.Client(project = project_id)

In [8]:
# wallet_to_wallet_edge_query = """
# SELECT
#     from_address,
#     to_address,
#     COUNT(*) as num_transactions,
#     SUM(value) as total_value_wei,
#     AVG(value) as avg_value_wei,
#     MIN(value) as min_value_wei,
#     MAX(value) as max_value_wei,
#     AVG(gas) as avg_gas_used,
#     AVG(gas_price) as avg_gas_price,
#     MIN(block_timestamp) as first_interaction,
#     MAX(block_timestamp) as last_interaction,
#     COUNT(DISTINCT DATE(block_timestamp)) as active_days
# FROM `bigquery-public-data.crypto_ethereum.transactions`
# WHERE block_timestamp BETWEEN '2024-01-01' AND '2024-01-31'
#     AND to_address IS NOT NULL
#     AND value > 0
# GROUP BY from_address, to_address
# HAVING num_transactions >= 2
# """

# edges_df = bq_client.query(wallet_to_wallet_edge_query).to_dataframe()
# drive.mount('/content/drive')

# output_filepath = '/content/drive/MyDrive/cs467-project/wallet_to_wallet_edges.csv'

# edges_df.to_csv(output_filepath, index=False)

In [9]:
# node_features_query = """
# WITH wallet_stats AS (
#   SELECT
#     from_address as wallet,
#     COUNT(*) as outgoing_tx_count,
#     SUM(value) as total_sent_wei,
#     AVG(value) as avg_sent_wei,
#     STDDEV(value) as stddev_sent_wei,
#     AVG(gas_price) as avg_gas_price,
#     COUNT(DISTINCT to_address) as unique_recipients,
#     COUNT(DISTINCT DATE(block_timestamp)) as active_days,
#     MIN(block_timestamp) as first_tx,
#     MAX(block_timestamp) as last_tx
#   FROM `bigquery-public-data.crypto_ethereum.transactions`
#   WHERE block_timestamp BETWEEN '2024-01-01' AND '2024-01-31'
#   GROUP BY from_address
# ),
# incoming_stats AS (
#   SELECT
#     to_address as wallet,
#     COUNT(*) as incoming_tx_count,
#     SUM(value) as total_received_wei,
#     AVG(value) as avg_received_wei,
#     COUNT(DISTINCT from_address) as unique_senders
#   FROM `bigquery-public-data.crypto_ethereum.transactions`
#   WHERE block_timestamp BETWEEN '2024-01-01' AND '2024-01-31'
#     AND to_address IS NOT NULL
#   GROUP BY to_address
# )
# SELECT
#   COALESCE(w.wallet, i.wallet) as wallet,
#   COALESCE(w.outgoing_tx_count, 0) as outgoing_tx_count,
#   COALESCE(w.total_sent_wei, 0) as total_sent_wei,
#   COALESCE(w.avg_sent_wei, 0) as avg_sent_wei,
#   COALESCE(w.stddev_sent_wei, 0) as stddev_sent_wei,
#   COALESCE(w.avg_gas_price, 0) as avg_gas_price,
#   COALESCE(w.unique_recipients, 0) as unique_recipients,
#   COALESCE(i.incoming_tx_count, 0) as incoming_tx_count,
#   COALESCE(i.total_received_wei, 0) as total_received_wei,
#   COALESCE(i.avg_received_wei, 0) as avg_received_wei,
#   COALESCE(i.unique_senders, 0) as unique_senders,
#   COALESCE(w.active_days, 0) as active_days,
#   w.first_tx,
#   w.last_tx
# FROM wallet_stats w
# FULL OUTER JOIN incoming_stats i ON w.wallet = i.wallet
# """

# edges_df = bq_client.query(node_features_query).to_dataframe()
# drive.mount('/content/drive')

# output_filepath = '/content/drive/MyDrive/cs467-project/node_features_query.csv'

# edges_df.to_csv(output_filepath, index=False)

In [10]:
# Daily ETH price data from kaggle
# https://www.kaggle.com/datasets/imranbukhari/comprehensive-ethusd-1d-data?select=ETHUSD_1d_Combined_Index.csv
eth_price_path = '/content/drive/MyDrive/cs467-project/ETHUSD_1d_Combined_Index.csv'
raw_df = pd.read_csv(eth_price_path)
print(raw_df.shape)
raw_df.head()

(3300, 6)


,Open time,Open,High,Low,Close,Volume
0,2016-09-29 00:00:00,13.31,13.39,13.10,13.11,48120.899558
1,2016-09-30 00:00:00,13.11,13.39,13.08,13.23,45501.339082
2,2016-10-01 00:00:00,13.23,13.31,13.09,13.22,41853.151043
3,2016-10-02 00:06:00,13.22,13.31,13.19,13.28,32058.247075
4,2016-10-03 00:00:00,13.28,13.56,13.25,13.51,48398.396026


In [11]:
# Preprocess ETH price data
raw_df.columns = raw_df.columns.str.lower().str.strip()
eth_price_df = raw_df.rename(columns={'open time': 'date'})
eth_price_df = eth_price_df.sort_values('date').reset_index(drop=True)

# Calculate daily return
eth_price_df['daily_return'] = eth_price_df['close'].pct_change()

# Calculate cumulative return (from start)
eth_price_df['cumulative_return'] = (1 + eth_price_df['daily_return']).cumprod() - 1

start_date = '2024-01-01'
end_date = '2024-01-31'

eth_prices_filtered = eth_price_df[
    (eth_price_df['date'] >= start_date) &
    (eth_price_df['date'] <= end_date)
].copy()

eth_prices_filtered['date'] = pd.to_datetime(eth_prices_filtered['date'])
eth_prices_filtered['date'] = eth_prices_filtered['date'].dt.tz_localize(None).dt.normalize()

eth_prices_filtered.head()

,date,open,high,low,close,volume,daily_return,cumulative_return
2650,2024-01-01,2281.61,2354.53,2266.77,2352.46,8.866700e+05,0.031053,178.440122
2651,2024-01-02,2352.46,2434.01,2342.11,2356.50,2.005903e+06,0.001717,178.748284
2652,2024-01-03,2356.50,2388.15,2075.14,2210.03,3.429723e+06,-0.062156,167.575896
2653,2024-01-04,2210.03,2297.79,2202.57,2268.36,1.386424e+06,0.026393,172.025172
2654,2024-01-05,2268.36,2279.24,2208.05,2268.94,1.422841e+06,0.000256,172.069413


In [12]:
# labels top x percentile of performers with minimum y transactions of next N days return
def compute_wallet_performance(edges_df, eth_prices_filtered, x, y, N):
    edges_df = edges_df.copy()

    # Timezone cleaning
    edges_df['first_interaction'] = pd.to_datetime(edges_df['first_interaction'])
    edges_df['first_interaction'] = (
            edges_df['first_interaction'].dt.tz_convert('UTC')
                            .dt.tz_localize(None)
                            .dt.normalize()
    )

    edges_df['last_interaction'] = pd.to_datetime(edges_df['last_interaction'])
    edges_df['last_interaction'] = (
            edges_df['last_interaction'].dt.tz_convert('UTC')
                            .dt.tz_localize(None)
                            .dt.normalize()
    )

    #Calculating hyperparameter N day return
    forward_days = N
    eth_prices_filtered = eth_prices_filtered.copy()
    eth_prices_filtered['forward_return'] = eth_prices_filtered['close'].shift(-forward_days) / eth_prices_filtered['close'] - 1
    forward_returns = eth_prices_filtered.set_index('date')['forward_return'].to_dict()

    wallet_metrics = {}

    for _, row in edges_df.iterrows():
        to_wallet = row['to_address']
        from_wallet = row['from_address']

        # Use first_interaction as the transaction date for now
        tx_date = row['first_interaction']
        tx_date = pd.Timestamp(tx_date).normalize()
        value_wei = float(row['total_value_wei'])

        # Get forward return for this date
        fwd_ret = forward_returns.get(tx_date, 0)

        # Initialize wallet metrics
        for wallet in [to_wallet, from_wallet]:
            if wallet not in wallet_metrics:
                wallet_metrics[wallet] = {
                    'total_received_value': 0,
                    'total_sent_value': 0,
                    'weighted_receive_return': 0,
                    'weighted_send_return': 0,
                    'tx_count': 0,
                    'first_tx_date': tx_date,
                    'last_tx_date': tx_date
                  }
        # Metrics for receiver (positive timing if received before price increase)
        wallet_metrics[to_wallet]['total_received_value'] += value_wei
        wallet_metrics[to_wallet]['weighted_receive_return'] += value_wei * fwd_ret
        wallet_metrics[to_wallet]['tx_count'] += 1
        wallet_metrics[to_wallet]['last_tx_date'] = max(wallet_metrics[to_wallet]['last_tx_date'], tx_date)

        #Metrics for sender
        wallet_metrics[from_wallet]['total_sent_value'] += value_wei
        wallet_metrics[from_wallet]['weighted_send_return'] -= value_wei * fwd_ret  #negative because selling
        wallet_metrics[from_wallet]['tx_count'] += 1
        wallet_metrics[from_wallet]['last_tx_date'] = max(wallet_metrics[from_wallet]['last_tx_date'], tx_date)

    wallet_perf_df = pd.DataFrame.from_dict(wallet_metrics, orient='index')
    wallet_perf_df.index.name = 'wallet'
    wallet_perf_df = wallet_perf_df.reset_index()

    wallet_perf_df = wallet_perf_df[wallet_perf_df['tx_count'] >= y] #avoid lucky wallets with low trade count, want active wallets

    '''
    # code for computing combined buy and sell alpha
    total_value = wallet_perf_df['total_received_value'] + wallet_perf_df['total_sent_value']
    total_value = total_value.replace(0, 1)  #avoid dividing by zero

    wallet_perf_df['timing_alpha'] = (
        wallet_perf_df['weighted_receive_return'] + wallet_perf_df['weighted_send_return']
    ) / total_value

    wallet_perf_df['timing_alpha_weighted'] = (
    wallet_perf_df['timing_alpha'] * np.log1p(wallet_perf_df['tx_count'])
    )

    # Normalize timing alpha
    wallet_perf_df['timing_alpha_normalized'] = (
        wallet_perf_df['timing_alpha_weighted'] - wallet_perf_df['timing_alpha_weighted'].mean()
    ) / (wallet_perf_df['timing_alpha_weighted'].std() + 1e-8)

    threshold = np.percentile(wallet_perf_df['timing_alpha_normalized'], x)
    wallet_perf_df['is_smart_money'] = (wallet_perf_df['timing_alpha_normalized'] >= threshold).astype(int)
    '''

    # Compute separate buyer and seller alpha
    wallet_perf_df['buyer_alpha'] = (
    wallet_perf_df['weighted_receive_return'] /
    wallet_perf_df['total_received_value'].replace(0, 1)
    )
    # Testing multiply alpha by log(#trades) so that it avoids overfitting to lower numbers of trades and rewards people with high tx_count and high alpha
    wallet_perf_df['buyer_alpha_weighted'] = (
    wallet_perf_df['buyer_alpha'] * np.log1p(wallet_perf_df['tx_count'])
    )
    wallet_perf_df['buyer_alpha_normalized'] = (
        wallet_perf_df['buyer_alpha_weighted'] - wallet_perf_df['buyer_alpha_weighted'].mean()
    ) / (wallet_perf_df['buyer_alpha_weighted'].std() + 1e-8)


    wallet_perf_df['seller_alpha'] = (
    wallet_perf_df['weighted_send_return'] /
    wallet_perf_df['total_sent_value'].replace(0, 1)
    )
    wallet_perf_df['seller_alpha_weighted'] = (
    wallet_perf_df['seller_alpha'] * np.log1p(wallet_perf_df['tx_count'])
    )
    wallet_perf_df['seller_alpha_normalized'] = (
        wallet_perf_df['seller_alpha_weighted'] - wallet_perf_df['seller_alpha_weighted'].mean()
    ) / (wallet_perf_df['seller_alpha_weighted'].std() + 1e-8)

    # Now label "smart buyer/seller" by alpha threshold
    # if x is 80, this means top 20% are "smart"
    buyer_mask = wallet_perf_df['total_received_value'] > 0 #for buyers, dont include if never bought
    buyer_threshold = np.percentile(wallet_perf_df.loc[buyer_mask, 'buyer_alpha_normalized'].dropna(), x)
    seller_mask = wallet_perf_df['total_sent_value'] > 0
    seller_threshold = np.percentile( wallet_perf_df.loc[seller_mask, 'seller_alpha_normalized'].dropna(), x)

    wallet_perf_df['is_smart_buyer'] = (wallet_perf_df['buyer_alpha_normalized'] >= buyer_threshold).astype(int)
    wallet_perf_df['is_smart_seller'] = (wallet_perf_df['seller_alpha_normalized'] >= seller_threshold).astype(int)

    print(f"{len(wallet_perf_df)} total wallets")
    print(f"Hyperparameters: {x}th-percentile, {N}-day return, minimum {y} trades")
    print(f"Smart buyer wallets: {wallet_perf_df['is_smart_buyer'].sum()}, threshold: {buyer_threshold}")
    print(f"Smart seller wallets: {wallet_perf_df['is_smart_seller'].sum()}, threshold: {seller_threshold}")
    both_mask = (
    (wallet_perf_df['is_smart_buyer'] == 1) &
    (wallet_perf_df['is_smart_seller'] == 1)
    )
    num_both = both_mask.sum()
    print(f"Smart buyer AND seller wallets: {num_both} \n")

    print(wallet_perf_df['buyer_alpha_normalized'].describe())
    print(wallet_perf_df['seller_alpha_normalized'].describe())

    return wallet_perf_df

In [13]:
wallet_to_wallet_edge_path = '/content/drive/MyDrive/cs467-project/wallet_to_wallet_edges.csv'
node_features_path = '/content/drive/MyDrive/cs467-project/node_features_query.csv'

edges_df = pd.read_csv(wallet_to_wallet_edge_path, nrows=100000)
nodes_df = pd.read_csv(node_features_path, nrows=100000)

# remove self loops
edges_df = edges_df[edges_df['from_address'] != edges_df['to_address']]

edges_df['num_transactions'] = pd.to_numeric(edges_df['num_transactions'], errors='coerce')
edges_df['total_value_wei'] = pd.to_numeric(edges_df['total_value_wei'], errors='coerce')
edges_df['avg_value_wei'] = pd.to_numeric(edges_df['avg_value_wei'], errors='coerce')

# Fill any NaN values with 0
edges_df[['num_transactions', 'total_value_wei', 'avg_value_wei']] = \
    edges_df[['num_transactions', 'total_value_wei', 'avg_value_wei']].fillna(0)

/tmp/ipython-input-896387691.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(wallet_to_wallet_edge_path, nrows=100000)


In [14]:
wallet_perf_df = compute_wallet_performance(edges_df, eth_prices_filtered, 80, 3, 7)

1999 total wallets
Hyperparameters: 80th-percentile, 7-day return, minimum 3 trades
Smart buyer wallets: 173, threshold: 1.0982698964928086
Smart seller wallets: 138, threshold: 0.5716317386666102
Smart buyer AND seller wallets: 10 

count    1.408000e+03
mean    -4.037175e-17
std      9.999999e-01
min     -3.831807e+00
25%     -2.627143e-01
50%     -1.593942e-01
75%     -1.593942e-01
max      7.603513e+00
Name: buyer_alpha_normalized, dtype: float64
count    1.418000e+03
mean    -5.010880e-18
std      9.999999e-01
min     -8.153753e+00
25%      1.954595e-01
50%      1.954595e-01
75%      2.782251e-01
max      3.684598e+00
Name: seller_alpha_normalized, dtype: float64


In [15]:
# example smart buyers
wallet_perf_df[wallet_perf_df['is_smart_buyer'] == 1].head()

,wallet,total_received_value,total_sent_value,weighted_receive_return,weighted_send_return,tx_count,first_tx_date,last_tx_date,buyer_alpha,buyer_alpha_weighted,buyer_alpha_normalized,seller_alpha,seller_alpha_weighted,seller_alpha_normalized,is_smart_buyer,is_smart_seller
36,0x5d738fbf1d8940bbe72af847d88c517064de76e7,1.082975e+16,1.105912e+20,1.836524e+15,9.708378e+17,6,2024-01-01,2024-01-22,0.169581,0.329990,2.749434,0.008779,0.017082,0.399009,1,0
37,0x974caa59e49682cda0ad2bbe82983419a2ecc400,7.673120e+21,6.089065e+21,2.679293e+20,NaN,887,2024-01-02,2024-01-30,0.034918,0.237057,1.930236,NaN,NaN,NaN,1,0
56,0x22bda31bc03cfc8c0b8f56743b4b34a8ec9d7579,7.557913e+19,2.832270e+18,5.993815e+18,NaN,8,2024-01-01,2024-01-29,0.079305,0.174251,1.376612,NaN,NaN,NaN,1,0
160,0xa83114a443da1cecefc50368531cace9f37fcccb,5.106001e+20,4.736802e+19,5.255003e+19,1.115342e+18,8,2024-01-12,2024-01-20,0.102918,0.226134,1.833956,0.023546,0.051737,0.811940,1,1
307,0xbb4d1dc5c1abec4ea11166ec97e714862863ad1d,2.152172e+21,1.926517e+19,3.649683e+20,NaN,14,2024-01-03,2024-01-26,0.169581,0.459235,3.888714,NaN,NaN,NaN,1,0


In [16]:
# example both smart buyer and seller
wallet_perf_df[(wallet_perf_df['is_smart_buyer'] == 1) & (wallet_perf_df['is_smart_seller'] == 1)].head()

,wallet,total_received_value,total_sent_value,weighted_receive_return,weighted_send_return,tx_count,first_tx_date,last_tx_date,buyer_alpha,buyer_alpha_weighted,buyer_alpha_normalized,seller_alpha,seller_alpha_weighted,seller_alpha_normalized,is_smart_buyer,is_smart_seller
160,0xa83114a443da1cecefc50368531cace9f37fcccb,5.106001e+20,4.736802e+19,5.255003e+19,1.115342e+18,8,2024-01-12,2024-01-20,0.102918,0.226134,1.833956,0.023546,0.051737,0.811940,1,1
7862,0x7e6c00d943c634b08056f941217c86083203bd37,1.481270e+18,2.404247e+19,1.659536e+17,7.865766e+17,3,2024-01-13,2024-01-13,0.112035,0.155313,1.209674,0.032716,0.045354,0.735890,1,1
9137,0xadb86093d50576b938e2400beadd1e7ed58bd6be,2.168762e+17,2.754437e+17,2.466190e+16,1.565435e+16,3,2024-01-03,2024-01-11,0.113714,0.157641,1.230198,0.056833,0.078788,1.134274,1,1
13371,0x921067eed49feb11202809e5877ce9bcb0df26c6,1.307392e+18,1.102355e+19,1.345538e+17,8.567313e+17,9,2024-01-14,2024-01-21,0.102918,0.236977,1.929530,0.077718,0.178953,2.327822,1,1
17556,0xce9374ee5a03ed88bc97920ac62bc1e35e377750,2.195274e+19,1.146322e+19,2.483924e+18,6.514913e+17,3,2024-01-07,2024-01-11,0.113149,0.156857,1.223287,0.056833,0.078788,1.134274,1,1


In [17]:
unique_wallets = pd.concat([
    edges_df['from_address'],
    edges_df['to_address']
]).unique()

wallet_to_idx = {wallet: idx for idx, wallet in enumerate(unique_wallets)}

# Build edge index
edge_index = torch.tensor([
    [wallet_to_idx[addr] for addr in edges_df['from_address']],
    [wallet_to_idx[addr] for addr in edges_df['to_address']]
], dtype=torch.long)

# Build edge attributes
edge_attr = torch.tensor(
    edges_df[['num_transactions', 'total_value_wei', 'avg_value_wei']].values,
    dtype=torch.float32
)

# Merge wallet perf stats into node table
wallet_perf_df = wallet_perf_df.copy()
# this is for the train/test splitting
wallet_perf_df['last_tx_date'] = pd.to_datetime(wallet_perf_df['last_tx_date'])
perf_cols = [
    'wallet',
    'last_tx_date',
    'is_smart_buyer',
    'is_smart_seller',
    'buyer_alpha_normalized',
    'seller_alpha_normalized',
    'tx_count'
]

wallet_perf_df = wallet_perf_df.reset_index()
perf_subset = wallet_perf_df[perf_cols]
nodes_merged = nodes_df.merge(perf_subset, on='wallet', how='left')


nodes_merged.set_index('wallet', inplace=True)
nodes_dict = nodes_merged.to_dict('index')

# Build node features matrix (much faster)
feature_cols = ['outgoing_tx_count', 'incoming_tx_count', 'total_sent_wei',
                'total_received_wei', 'unique_recipients', 'unique_senders', 'active_days',
                'buyer_alpha_normalized', 'seller_alpha_normalized', 'tx_count'] # added performance features

node_features_list = []
for wallet in unique_wallets:
    row = nodes_dict.get(wallet)
    if row is None:
        node_features_list.append([0.0] * len(feature_cols))
    else:
        node_features_list.append([float(row.get(col, 0.0)) for col in feature_cols])

node_features = torch.tensor(node_features_list, dtype=torch.float)

# Normalize features
node_features = (node_features - node_features.mean(dim=0)) / (node_features.std(dim=0) + 1e-8)

# Align perf rows to unique_wallets
labels_aligned = nodes_merged.reindex(unique_wallets)
# NaN labels are 0/non-smart
label_vals = labels_aligned[['is_smart_buyer', 'is_smart_seller']].fillna(0).values
y = torch.tensor(label_vals, dtype=torch.float) # shape: (num_nodes, 2)

In [18]:
#Train test split for data, temporal instead of random sampling
test_ratio = 0.2
num_nodes = len(unique_wallets)

last_dates = pd.to_datetime(labels_aligned['last_tx_date'])
sorted_wallets = last_dates.sort_values().index.tolist()

# size of splits
num_test  = int(num_nodes * test_ratio)
num_train = num_nodes - num_test

train_wallets = set(sorted_wallets[:num_train])
test_wallets  = set(sorted_wallets[num_train:])

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask  = torch.zeros(num_nodes, dtype=torch.bool)

for i, wallet in enumerate(unique_wallets):
    if wallet in train_wallets:
        train_mask[i] = True
    elif wallet in test_wallets:
        test_mask[i] = True
    # if last_tx_date is NaN, both false


# Create PyG Data object
data = Data(
    x=node_features,
    edge_index=edge_index,
    edge_attr=edge_attr,
    y=y,
    train_mask = train_mask,
    test_mask = test_mask
)

print(f"Num nodes: {data.num_nodes}")
print(f"Num edges: {data.num_edges}")
print(f"Node feature dim: {data.num_node_features}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self loops: {data.has_self_loops()}")
print(f"Is directed: {data.is_directed()}")

print(f"Train nodes: {int(train_mask.sum())}, Test nodes: {int(test_mask.sum())}")

Num nodes: 121932
Num edges: 99329
Node feature dim: 10
Has isolated nodes: False
Has self loops: False
Is directed: True
Train nodes: 97546, Test nodes: 24386
